In [2]:
import os
os.environ["TRANSFORMERS_NO_TF"] = "1"

from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments
from datasets import Dataset


In [24]:
!pip uninstall -y keras tensorflow tf-keras tensorflow-estimator


Found existing installation: keras 3.10.0
Uninstalling keras-3.10.0:
  Successfully uninstalled keras-3.10.0
Found existing installation: tensorflow 2.19.0
Uninstalling tensorflow-2.19.0:
  Successfully uninstalled tensorflow-2.19.0
Found existing installation: tf_keras 2.19.0
Uninstalling tf_keras-2.19.0:
  Successfully uninstalled tf_keras-2.19.0


In [9]:
!pip install transformers datasets accelerate

In [4]:
import pandas as pd

df = pd.read_csv('for_training_no_nulls.csv')

# Clean and prepare
df.dropna(subset=['title', 'description', 'genre', 'rating'], inplace=True)

df['rating'] = df['rating'].astype(str)
df['genre'] = df['genre'].str.lower().str.strip()

# Create task inputs
df_rating = df.copy()
df_rating['input_text'] = "predict rating: Title: " + df['title'] + " Description: " + df['description']
df_rating['target_text'] = df['rating']

df_genre = df.copy()
df_genre['input_text'] = "predict genre: Title: " + df['title'] + " Description: " + df['description']
df_genre['target_text'] = df['genre']

# Combine both tasks
final_df = pd.concat([df_rating[['input_text', 'target_text']], df_genre[['input_text', 'target_text']]])
final_df = final_df.sample(frac=1).reset_index(drop=True)


In [20]:
!pip install sentencepiece

In [22]:
!pip install tf-keras

  Using cached tf_keras-2.19.0-py3-none-any.whl.metadata (1.8 kB)
  Using cached tensorflow-2.19.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl.metadata (4.1 kB)
Using cached tf_keras-2.19.0-py3-none-any.whl (1.7 MB)
Using cached tensorflow-2.19.0-cp311-cp311-manylinux_2_17_x86_64.manylinux2014_x86_64.whl (644.9 MB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.4/1.4 MB 7.7 MB/s eta 0:00:00


In [5]:
pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cpu


Looking in indexes: https://download.pytorch.org/whl/cpu
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 176.0/176.0 MB 142.5 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 23.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 30.5 MB/s eta 0:00:00
  Attempting uninstall: torch
    Found existing installation: torch 2.4.1
    Uninstalling torch-2.4.1:
      Successfully uninstalled torch-2.4.1
  Attempting uninstall: torchvision
    Found existing installation: torchvision 0.19.1
    Uninstalling torchvision-0.19.1:
      Successfully uninstalled torchvision-0.19.1
  Attempting uninstall: torchaudio
    Found existing installation: torchaudio 2.4.1
    Uninstalling torchaudio-2.4.1:
      Successfully uninstalled torchaudio-2.4.1
Note: you may need to restart the kernel to use updated packages.


In [ ]:
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments, DataCollatorForSeq2Seq
from datasets import Dataset

# Load tokenizer and model
tokenizer = T5Tokenizer.from_pretrained("t5-small")
model = T5ForConditionalGeneration.from_pretrained("t5-small")
model = model.to("cuda")

# Tokenize dataset
def preprocess(example):
    input_enc = tokenizer(example['input_text'], padding="max_length", truncation=True, max_length=512)
    target_enc = tokenizer(example['target_text'], padding="max_length", truncation=True, max_length=64)
    return {
        'input_ids': input_enc['input_ids'],
        'attention_mask': input_enc['attention_mask'],
        'labels': target_enc['input_ids']
    }

dataset = Dataset.from_pandas(final_df)
tokenized_dataset = dataset.map(preprocess)

# Define training args
training_args = TrainingArguments(
    output_dir="./t5-multitask",
    per_device_train_batch_size=4,
    num_train_epochs=3,
    logging_steps=10,
    save_total_limit=1,
    save_steps=100,
    fp16=True,                  #  Enables fast GPU mixed precision
    dataloader_pin_memory=True #  Helps with GPU data loading
)

data_collator = DataCollatorForSeq2Seq(tokenizer=tokenizer, model=model)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_dataset,
    data_collator=data_collator,
)


# Train
trainer.train()

Map:   0%|          | 0/102280 [00:00<?, ? examples/s]

Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
Passing a tuple of `past_key_values` is deprecated and will be removed in Transformers v4.48.0. You should pass an instance of `EncoderDecoderCache` instead, e.g. `past_key_values=EncoderDecoderCache.from_legacy_cache(past_key_values)`.


Step,Training Loss
10,13.889200
20,6.816100
30,2.930200
40,2.008200
50,1.648800
60,1.444500
70,1.524100
80,1.144700
90,1.362800
100,0.971100


In [3]:
import torch
print("CUDA Available:", torch.cuda.is_available())
print("GPU Name:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU")

CUDA Available: False
GPU Name: No GPU


In [5]:
!pip install --upgrade torch torchvision torchaudio --index-url https://download.pytorch.org/whl/cu118


Looking in indexes: https://download.pytorch.org/whl/cu118
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 23.2/23.2 MB 54.4 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 875.6/875.6 kB 8.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.1/13.1 MB 58.3 MB/s eta 0:00:0000:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 663.9/663.9 MB 87.3 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 417.9/417.9 MB 132.4 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 168.4/168.4 MB 125.8 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.1/58.1 MB 100.9 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 128.2/128.2 MB 130.5 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 204.1/204.1 MB 144.7 MB/s eta 0:00:0000:0100:01
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 147.8/147.8 MB 131.2 MB/s eta 0:00:0000:0100:01
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 905.3/9

In [1]:
import torch
print("CUDA available:", torch.cuda.is_available())
print("GPU:", torch.cuda.get_device_name(0) if torch.cuda.is_available() else "No GPU found")


CUDA available: True
GPU: Tesla V100S-PCIE-32GB
